In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNLSTM(nn.Module):
    def __init__(self, sequence_length, num_features, num_classes):
        super(CNNLSTM, self).__init__()
        # CNN layers
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3, padding=1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.batchnorm3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.lstm_input_features = self._calculate_lstm_input_features(sequence_length)

        self.lstm_hidden_size = 128
        self.num_lstm_layers = 7
        self.lstm = nn.LSTM(input_size=self.lstm_input_features, hidden_size=self.lstm_hidden_size, num_layers=self.num_lstm_layers, batch_first=True)

        # Fully connected layers
        self.fc1 = nn.Linear(self.lstm_hidden_size, 128)
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(64, num_classes)

    def _calculate_lstm_input_features(self, sequence_length):
        sample = torch.autograd.Variable(torch.rand(1, 1, sequence_length))
        sample = self.pool1(F.relu(self.batchnorm1(self.conv1(sample))))
        sample = self.pool2(F.relu(self.batchnorm2(self.conv2(sample))))
        sample = self.pool3(F.relu(self.batchnorm3(self.conv3(sample))))
        return sample.numel()
   

    def forward(self, x):
        # Reshape input to (batch_size, num_features, sequence_length)
        x = x.view(x.size(0), 1, -1)

        # Apply CNN layers
        x = self.pool1(F.relu(self.batchnorm1(self.conv1(x))))
        x = self.pool2(F.relu(self.batchnorm2(self.conv2(x))))
        x = self.pool3(F.relu(self.batchnorm3(self.conv3(x))))

        # Prepare for LSTM - Flatten and reshape
        x = x.view(x.size(0), -1, self.lstm_input_features)

        # Apply LSTM
        lstm_output, (hn, cn) = self.lstm(x)

        # Select the output of the last LSTM time step
        x = lstm_output[:, -1, :]

        # Apply Fully connected layers
        x = F.relu(self.fc1(self.dropout1(x)))
        x = F.relu(self.fc2(self.dropout2(x)))
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)




sequence_length = 128  # Sequence length of each sample
num_features = 1      # Number of features per sequence element
num_classes = 9      # Number of output classes
model = CNNLSTM(sequence_length, num_features, num_classes)

# Print the model
print(model)

CNNLSTM(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (batchnorm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (lstm): LSTM(4096, 128, num_layers=7, batch_first=True)
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (